# Fully Connected Deep Neural Network
## 预处理

In [1]:
from load import load_all_dataset
X_train, y_train, X_test, y_test = load_all_dataset()
import numpy as np
np.set_printoptions(formatter={"float":lambda x: "{:.3f}".format(x)})

Train data
Optical Dataset composed of
46110 source samples
50862 source background samples
438 target labeled samples
8202 target unlabeled samples
29592 target background samples
 Optical Dataset labels composed of
46110 labels of source samples
438 labels of target samples

Test data
Optical Dataset composed of
0 source samples
0 source background samples
17758 target labeled samples
0 target unlabeled samples
47275 target background samples
 Optical Dataset labels composed of
0 labels of source samples
17758 labels of target samples



In [2]:
import tensorflow as tf
from tensorflow.keras import layers

In [3]:
# 去除NaN
from feature_extractor import FeatureExtractor
from numpy import newaxis
class FE(FeatureExtractor):
    def transform(self, X):
        np.nan_to_num(X, copy=False)
        X = X.reshape(X.shape[0], -1)
        return X[:,:,newaxis]

fe = FE()
X_train.source = fe.transform(X_train.source)
X_train.target = fe.transform(X_train.target)
X_test.target = fe.transform(X_test.target)
X_train.target.shape

(438, 6720, 1)

In [16]:
# 将数据集转换为TensorFlow格式
train_dataset = tf.data.Dataset.from_tensor_slices((X_train.source, y_train.source)).batch(16)
valid_dataset = tf.data.Dataset.from_tensor_slices((X_train.target, y_train.target)).batch(16)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test.target, y_test.target))

# 额外操作
train_dataset = train_dataset.map( lambda x, y: (tf.image.random_flip_left_right(x), y) ) # array must be 3D
train_dataset = train_dataset.repeat()
valid_dataset = valid_dataset.repeat()

AttributeError: 'RepeatDataset' object has no attribute 'shape'

## 搭建网络模型
参考资料

1. [Build your first Neural Network in TensorFlow 2](https://towardsdatascience.com/building-your-first-neural-network-in-tensorflow-2-tensorflow-for-hackers-part-i-e1e2f1dfe7a0)

In [11]:
# Sequential groups a linear stack of layers into a tf.keras.Model.
# https://www.tensorflow.org/api_docs/python/tf/keras/Sequential
model = tf.keras.Sequential()
model.add( layers.Flatten(input_shape=(6720,), name="Input_Layer") )

num_fully_connected_layers = 10
for i in range(num_fully_connected_layers):
    model.add( layers.Dense(256, activation="relu", name="Layer{}".format(i+1)) )

model.add( layers.Dropout(0.5, name="Layer-1"))
model.add( layers.Dense(1, activation='softmax', name="Output_Layer") )
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_Layer (Flatten)        (None, 6720)              0         
_________________________________________________________________
Layer1 (Dense)               (None, 256)               1720576   
_________________________________________________________________
Layer2 (Dense)               (None, 256)               65792     
_________________________________________________________________
Layer3 (Dense)               (None, 256)               65792     
_________________________________________________________________
Layer4 (Dense)               (None, 256)               65792     
_________________________________________________________________
Layer5 (Dense)               (None, 256)               65792     
_________________________________________________________________
Layer6 (Dense)               (None, 256)              

## 训练模型
'categorical_crossentropy' is much better than 'binary_crossentropy'

In [12]:
import datetime as dt
callbacks = [
  # Write TensorBoard logs to `./logs` directory
  tf.keras.callbacks.TensorBoard(log_dir='./log/{}'.format(
      dt.datetime.now().strftime("%Y-%m-%d-%H-%M-%S DNN")), write_images=True),
  ]
model.compile(optimizer="adam",
              loss='categorical_crossentropy',
              metrics=['acc'])
model.fit(train_dataset, epochs=100, steps_per_epoch=200,
          validation_data=valid_dataset,
          validation_steps=3, callbacks=callbacks)

Epoch 1/100
200/200 [==============================] - 2s 8ms/step - loss: nan - acc: 0.3525 - val_loss: nan - val_acc: 0.7083
Epoch 2/100
200/200 [==============================] - 1s 6ms/step - loss: nan - acc: 0.9084 - val_loss: nan - val_acc: 0.7083
Epoch 3/100
200/200 [==============================] - 1s 6ms/step - loss: nan - acc: 0.9203 - val_loss: nan - val_acc: 0.7083
Epoch 4/100
200/200 [==============================] - 1s 6ms/step - loss: nan - acc: 0.9172 - val_loss: nan - val_acc: 0.7083
Epoch 5/100
200/200 [==============================] - 1s 6ms/step - loss: nan - acc: 0.9209 - val_loss: nan - val_acc: 0.7083
Epoch 6/100
200/200 [==============================] - 1s 6ms/step - loss: nan - acc: 0.9256 - val_loss: nan - val_acc: 0.7083
Epoch 7/100
200/200 [==============================] - 1s 6ms/step - loss: nan - acc: 0.9219 - val_loss: nan - val_acc: 0.7083
Epoch 8/100
200/200 [==============================] - 1s 6ms/step - loss: nan - acc: 0.9084 - val_loss: nan - 

## 预测概率

In [20]:
print(X_test.target.shape)
X = X_test.target.reshape(X_test.target.shape[0], -1)
print(X.shape)
model.predict(X)

(17758, 6720, 1)
(17758, 6720)


array([[nan],
       [nan],
       [nan],
       ...,
       [nan],
       [nan],
       [nan]], dtype=float32)

## 查看Tensorboard

In [14]:
%tensorboard

UsageError: Line magic function `%tensorboard` not found.
